In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda:0


In [ ]:
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class AcceleratedCNN(nn.Module):
    def __init__(self):
        super(AcceleratedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:02<00:00, 62796729.54it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
def train_model(model, trainloader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    start_time = time.time()

    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if i % 2000 == 1999:
                print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0
    end_time = time.time()
    print('Finished Training')
    print(f'Training took {end_time - start_time:.2f} seconds')

In [ ]:
def test_model(model, testloader):
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    end_time = time.time()
    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')
    print(f'Testing took {end_time - start_time:.2f} seconds')

In [ ]:
basic_model = BasicCNN().to(device)
train_model(basic_model, trainloader)
test_model(basic_model, testloader)

[1, 2000] loss: 2.010
[1, 4000] loss: 1.626
[1, 6000] loss: 1.440
[1, 8000] loss: 1.336
[1, 10000] loss: 1.284
[1, 12000] loss: 1.188
[2, 2000] loss: 1.098
[2, 4000] loss: 1.064
[2, 6000] loss: 1.072
[2, 8000] loss: 1.050
[2, 10000] loss: 1.008
[2, 12000] loss: 0.991
[3, 2000] loss: 0.893
[3, 4000] loss: 0.875
[3, 6000] loss: 0.880
[3, 8000] loss: 0.884
[3, 10000] loss: 0.871
[3, 12000] loss: 0.861
[4, 2000] loss: 0.734
[4, 4000] loss: 0.755
[4, 6000] loss: 0.752
[4, 8000] loss: 0.757
[4, 10000] loss: 0.744
[4, 12000] loss: 0.774
[5, 2000] loss: 0.599
[5, 4000] loss: 0.617
[5, 6000] loss: 0.633
[5, 8000] loss: 0.632
[5, 10000] loss: 0.660
[5, 12000] loss: 0.685
[6, 2000] loss: 0.478
[6, 4000] loss: 0.532
[6, 6000] loss: 0.553
[6, 8000] loss: 0.542
[6, 10000] loss: 0.574
[6, 12000] loss: 0.579
[7, 2000] loss: 0.390
[7, 4000] loss: 0.417
[7, 6000] loss: 0.452
[7, 8000] loss: 0.470
[7, 10000] loss: 0.487
[7, 12000] loss: 0.504
[8, 2000] loss: 0.332
[8, 4000] loss: 0.338
[8, 6000] loss: 0.

In [ ]:
accel_model = AcceleratedCNN().to(device)
train_model(accel_model, trainloader)
test_model(accel_model, testloader)

[1, 2000] loss: 2.005
[1, 4000] loss: 1.632
[1, 6000] loss: 1.492
[1, 8000] loss: 1.430
[1, 10000] loss: 1.377
[1, 12000] loss: 1.310
[2, 2000] loss: 1.222
[2, 4000] loss: 1.175
[2, 6000] loss: 1.157
[2, 8000] loss: 1.132
[2, 10000] loss: 1.134
[2, 12000] loss: 1.086
[3, 2000] loss: 0.998
[3, 4000] loss: 1.006
[3, 6000] loss: 1.005
[3, 8000] loss: 1.003
[3, 10000] loss: 0.983
[3, 12000] loss: 0.972
[4, 2000] loss: 0.856
[4, 4000] loss: 0.876
[4, 6000] loss: 0.882
[4, 8000] loss: 0.906
[4, 10000] loss: 0.896
[4, 12000] loss: 0.913
[5, 2000] loss: 0.776
[5, 4000] loss: 0.795
[5, 6000] loss: 0.798
[5, 8000] loss: 0.798
[5, 10000] loss: 0.795
[5, 12000] loss: 0.828
[6, 2000] loss: 0.672
[6, 4000] loss: 0.709
[6, 6000] loss: 0.702
[6, 8000] loss: 0.735
[6, 10000] loss: 0.743
[6, 12000] loss: 0.780
[7, 2000] loss: 0.610
[7, 4000] loss: 0.625
[7, 6000] loss: 0.657
[7, 8000] loss: 0.671
[7, 10000] loss: 0.700
[7, 12000] loss: 0.693
[8, 2000] loss: 0.515
[8, 4000] loss: 0.571
[8, 6000] loss: 0.